In [1]:
import redis

# Підключення до Redis
# За замовчуванням: host='localhost', port=6379, db=0
r = redis.Redis(host='localhost', port=6379, db=0)

# Перевірка підключення
try:
    r.ping()
    print("Успішно підключено до Redis!")
except redis.exceptions.ConnectionError as e:
    print(f"Помилка підключення до Redis: {e}")

Успішно підключено до Redis!


In [ ]:
# Встановлення значення (SET key value)
r.set('mykey', 'Привіт, Redis!')
print(f"Значення 'mykey' встановлено.")

# Встановлення значення з часом життя (TTL) в секундах (SETEX key seconds value)
r.setex('temp_key', 60, 'Це тимчасове значення') # Зникне через 60 секунд
print(f"Значення 'temp_key' встановлено на 60 секунд.")

In [4]:
# Отримання значення (GET key)
value = r.get('mykey')
if value:
    print(f"Значення 'mykey': {value.decode('utf-8')}")
else:
    print(f"Ключ 'mykey' не знайдено.")

temp_value = r.get('temp_key')
if temp_value:
    print(f"Значення 'temp_key': {temp_value.decode('utf-8')}")
else:
    print(f"Ключ 'temp_key' не знайдено або термін дії закінчився.")

Значення 'mykey': Привіт, Redis!
Ключ 'temp_key' не знайдено або термін дії закінчився.


In [ ]:
# Видалення ключа (DEL key)
r.delete('mykey')
print(f"Ключ 'mykey' видалено.")

# Перевірка, що ключ більше не існує
deleted_value = r.get('mykey')
if deleted_value is None:
    print(f"Ключ 'mykey' більше не існує.")

Ключ 'mykey' видалено.
Ключ 'mykey' більше не існує.


In [9]:
# Збереження хешу (HSET hash_name field value)
r.hset('user:100', mapping={
    'name': 'Олена',
    'email': 'olena@example.com',
    'age': 30
})
print("Хеш 'user:100' збережено.")

# Отримання поля з хешу (HGET hash_name field)
user_name = r.hget('user:100', 'name')
if user_name:
    print(f"Ім'я користувача 100: {user_name.decode('utf-8')}")

# Отримання всіх полів хешу (HGETALL hash_name)
user_data = r.hgetall('user:100')
print("Всі дані користувача 100:")
for field, value in user_data.items():
    print(f"  {field.decode('utf-8')}: {value.decode('utf-8')}")

Хеш 'user:100' збережено.
Ім'я користувача 100: Олена
Всі дані користувача 100:
  name: Олена
  email: olena@example.com
  age: 30


In [10]:
# Додавання елементів у список справа (RPUSH list_name element1 element2 ...)
r.rpush('task_queue', 'Задача 1', 'Задача 2', 'Задача 3')
print("Додано задачі до 'task_queue'.")

# Отримання елементів зі списку зліва (LPOP list_name)
task = r.lpop('task_queue')
if task:
    print(f"Виконано задачу: {task.decode('utf-8')}")

# Отримання всіх елементів списку (LRANGE list_name start end)
all_tasks = r.lrange('task_queue', 0, -1)
print("Залишилось задач:")
for t in all_tasks:
    print(f"  {t.decode('utf-8')}")

Додано задачі до 'task_queue'.
Виконано задачу: Задача 1
Залишилось задач:
  Задача 2
  Задача 3


In [11]:
# Додавання елементів до множини (SADD set_name member1 member2 ...)
r.sadd('unique_visitors', 'user_a', 'user_b', 'user_a', 'user_c') # 'user_a' додасться лише раз
print("Додано відвідувачів до 'unique_visitors'.")

# Отримання всіх членів множини (SMEMBERS set_name)
visitors = r.smembers('unique_visitors')
print("Унікальні відвідувачі:")
for v in visitors:
    print(f"  {v.decode('utf-8')}")

# Перевірка наявності члена в множині (SISMEMBER set_name member)
if r.sismember('unique_visitors', 'user_b'):
    print("user_b є серед унікальних відвідувачів.")

Додано відвідувачів до 'unique_visitors'.
Унікальні відвідувачі:
  user_c
  user_a
  user_b
user_b є серед унікальних відвідувачів.


In [12]:
def get_data_from_db(item_id):
    """Імітація отримання даних з бази даних"""
    print(f"Отримуємо дані для item_id={item_id} з бази даних...")
    # У реальному застосунку тут був би виклик до БД
    return f"Дані для Item {item_id} з БД"

def get_item_data(item_id):
    cache_key = f"item_data:{item_id}"
    
    # Спроба отримати дані з кешу
    cached_data = r.get(cache_key)
    if cached_data:
        print(f"Отримано дані для item_id={item_id} з кешу.")
        return cached_data.decode('utf-8')
    
    # Якщо даних немає в кеші, отримуємо їх з БД
    data = get_data_from_db(item_id)
    
    # Зберігаємо дані в кеші з TTL (наприклад, 300 секунд = 5 хвилин)
    r.setex(cache_key, 300, data)
    print(f"Дані для item_id={item_id} збережено в кеші.")
    return data

print(get_item_data(1)) # Перший виклик: з БД, кешується
print(get_item_data(1)) # Другий виклик: з кешу
print(get_item_data(2)) # Новий елемент: з БД, кешується

Отримуємо дані для item_id=1 з бази даних...
Дані для item_id=1 збережено в кеші.
Дані для Item 1 з БД
Отримано дані для item_id=1 з кешу.
Дані для Item 1 з БД
Отримуємо дані для item_id=2 з бази даних...
Дані для item_id=2 збережено в кеші.
Дані для Item 2 з БД


In [13]:
pipe = r.pipeline() # Створення пайплайну

# Додаємо команди до пайплайну
pipe.set('key1', 'value1')
pipe.set('key2', 'value2')
pipe.get('key1')
pipe.delete('key2')
pipe.sadd('myset', 'a', 'b', 'c')

# Виконуємо всі команди за один раз
results = pipe.execute()
print("Результати виконання пайплайну:")
for res in results:
    print(res)

Результати виконання пайплайну:
True
True
b'value1'
1
3


In [16]:
# Транзакція з WATCH (для оптимістичного блокування)
with r.pipeline() as pipe:
    while True:
        try:
            pipe.watch('mycounter') # Спостерігаємо за ключем
            current_value = pipe.get('mycounter')
            current_value = int(current_value.decode('utf-8')) if current_value else 0

            # Починаємо транзакцію
            pipe.multi()
            pipe.set('mycounter', current_value + 1)
            
            # Виконуємо транзакцію
            pipe.execute()
            print(f"Лічильник збільшено до: {current_value + 1}")
            break # Успішно завершено
        except redis.exceptions.WatchError:
            print("Конфлікт: ключ 'mycounter' змінився. Повторна спроба.")
            # Повторюємо операцію, якщо ключ змінився

Лічильник збільшено до: 3


In [17]:
import redis
import time

r = redis.Redis(host='localhost', port=6379, db=0)

def increment_page_views(page_id):
    key = f"page_views:{page_id}"
    
    # INCRBY збільшує значення ключа на задану величину (або на 1, якщо не вказано)
    # Якщо ключ не існує, він створюється зі значенням 0 перед інкрементом.
    count = r.incr(key)
    print(f"Перегляди сторінки '{page_id}': {count}")
    return count

def get_page_views(page_id):
    key = f"page_views:{page_id}"
    count = r.get(key)
    return int(count.decode('utf-8')) if count else 0

print("\n--- Лічильник переглядів ---")
increment_page_views("home")
increment_page_views("about")
increment_page_views("home")
increment_page_views("contact")
increment_page_views("home")

print(f"Всього переглядів 'home': {get_page_views('home')}")
print(f"Всього переглядів 'about': {get_page_views('about')}")

# Очистимо для наступного запуску (опціонально)
r.delete("page_views:home", "page_views:about", "page_views:contact")
print("\nЛічильники очищені.")


--- Лічильник переглядів ---
Перегляди сторінки 'home': 1
Перегляди сторінки 'about': 1
Перегляди сторінки 'home': 2
Перегляди сторінки 'contact': 1
Перегляди сторінки 'home': 3
Всього переглядів 'home': 3
Всього переглядів 'about': 1

Лічильники очищені.
